# Detection V2 Network Defintion


Detection V1 Network는 다음과 같이 구성되어 있다.

* detectionv2 = backbone + rpn 
  * backbone = body (Resetn50) + FPN
    * body = stem + layer1 + layer2 + layer3 + layer4
    * FPN = fpn_inner2 + fpn_layer2 + fpn_inner3 + fpn_layer3 + fpn_inner4 + fpn_layer4 + top_block
      * top_blocks = p6 + p7
  * rpn = anchor_generator + head + box_selector_test
      * head = cls_tower + bbox_tower + cls_logits + bbox_pre

----

## backbone

backbone은 기본적으로 ResNet50 을 body로 하고 FPN (Feature Pyramid Network)을 convolution layer의 feature map에 연결한 구조이다.

### body (ResNet50)

body를 구성하고 있는 ResNet50은 stem + layer1 + layer2 + layer3 + layer4로 이루어진다.
* stem은 layer0이라고도 한다.

### stem 레이어

* 관련 소스: [StemWithFixedBatchNormclass 소스분석](http://echo.etri.re.kr:8090/pages/viewpage.action?pageId=78087898#MaskRCNNBenchMarkPyTorch%EB%B2%84%EC%A0%84%EC%86%8C%EC%8A%A4%EC%BD%94%EB%93%9C%EB%B6%84%EC%84%9D-StemWithFixedBatchNormclass)

stem 레이어는 아래와 같은 레이어들이 순서대로 연결된다.
* convolution : conv1
    * kernel_size: 7, stride:2, input ch:3, output ch:64, bias: Flase
* batchnormal : bn1
    * output ch:64
* relu 
* maxpool2d 
    * kernel_size =3, strid=2, padding=1

PyTorch 코드에서 relu와 maxpool2d는 funtional을 사용하여 forward() 내부에서 호출하기 때문에 print(model)에서는 표시되지 않는다.

Lomin 소스가 기반으로 하고 있는 mask-rcnn에서 batchnormal (bn1)은 일반적인 batchnorm은 [FrozenBatchNorm2d()](https://github.com/facebookresearch/maskrcnn-benchmark/blob/master/maskrcnn_benchmark/layers/batch_norm.py)를 사용한다.
> 즉, stem 레이어에서의 output ch:64이다.
> BatchNorm2d where the batch statistics and the affine parameters are fixed

In [3]:
import torch

In [12]:
mean = torch.FloatTensor([1, 2, 3, 4, 5])
mean_after_reshape = mean.reshape(1, -1, 1, 1)

In [13]:
print(mean)

tensor([1., 2., 3., 4., 5.])


In [14]:
print(mean_after_reshape)

tensor([[[[1.]],

         [[2.]],

         [[3.]],

         [[4.]],

         [[5.]]]])


In [32]:
from caffe import layers as L, params as P

In [33]:
help(caffe.layers)


Help on Layers in module caffe.net_spec object:

class Layers(builtins.object)
 |  A Layers object is a pseudo-module which generates functions that specify
 |  layers; e.g., Layers().Convolution(bottom, kernel_size=3) will produce a Top
 |  specifying a 3x3 convolution applied to bottom.
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, name)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [34]:
n=caffe.NetSpec()

Help on Layers in module caffe.net_spec object:

class Layers(builtins.object)
 |  A Layers object is a pseudo-module which generates functions that specify
 |  layers; e.g., Layers().Convolution(bottom, kernel_size=3) will produce a Top
 |  specifying a 3x3 convolution applied to bottom.
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, name)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

